In [1]:
import pandas as pd
import numpy as np
import geopy.distance
import geopy.point
import math

In [2]:
def calculate_distance(df):

    distance = 0
    
    for long1, lat1, alt1, long2, lat2, alt2 in zip(df["GNSS_Longitude"][:-1], df["GNSS_Latitude"][:-1], df["GNSS_Altitude"][:-1],df["GNSS_Longitude"][1:], df["GNSS_Latitude"][1:],df["GNSS_Altitude"][1:]):

        distance2d = geopy.distance.geodesic((long1, lat1), (long2, lat2)).m
        distance += np.sqrt(distance2d**2 + (alt1 - alt2)**2)

    return distance

In [3]:
for day in ["G1", "P1", "P2"]:
    gate_coordinates_df = pd.read_csv(f"../data/{day}/gate_coordinates.csv")

    gates_distances_df = pd.DataFrame(columns=["Gate", "Next_gate", "Aerial_distance", "Lateral_distance"])

    number_of_gates = np.max(gate_coordinates_df["num"])

    print("Number of gates is", number_of_gates)

    for gate_index in range(1, number_of_gates - 1):
        gate_loc = gate_coordinates_df.to_numpy()[gate_index - 1][1:]
        next_gate_loc = gate_coordinates_df.to_numpy()[gate_index][1:]
        next_next_gate_loc = gate_coordinates_df.to_numpy()[gate_index + 1][1:]

        aerial_distance = geopy.distance.geodesic(gate_loc, next_gate_loc).m

        aerial_distance_g1_g2 = aerial_distance
        aerial_distance_g1_g3 = geopy.distance.geodesic(gate_loc, next_next_gate_loc).m
        aerial_distance_g2_g3 = geopy.distance.geodesic(next_gate_loc, next_next_gate_loc).m

        a2 = aerial_distance_g2_g3 ** 2
        b2 = aerial_distance_g1_g2 ** 2
        c2 = aerial_distance_g1_g3 ** 2

        alpha = math.acos((b2 + c2 - a2) /
                         (2 * aerial_distance_g1_g2 * aerial_distance_g1_g3))
    

        lateral_distance = math.sin(alpha) * aerial_distance_g1_g2

        gates_distances_df.loc[len(gates_distances_df)] = [int(gate_index), int(gate_index + 1), aerial_distance, lateral_distance]
    gates_distances_df.to_csv(f"../data/filtered/gates_filtering/gates_distances_{day}.csv", index=False)


Number of gates is 28
Number of gates is 21
Number of gates is 21
